# Stardist Segmentation (use your own models)

(Reference:https://github.com/mpicbg-csbd/stardist/blob/master/examples/other2D/export_imagej_rois.ipynb)

This macro will export **label images** and **ROI information** with your own trained medel.

Be sure to check the input images size are similar to model sourse images size

For example, if you enlarge imgs size 4 times to train the model (25x25pixel->100x100pixel), here you have to also enlarge your raw imgs size 4 times before segmentation. After segmentation, if you want the correct information, you have to shrink the label imgs and divide the ROI values by 4.

In [ ]:
##Install packages which are not included in Google Colab.
!pip install tifffile # contains tools to operate tiff-files
!pip install csbdeep  # contains tools for restoration of fluorescence microcopy images (Content-aware Image Restoration, CARE). It uses Keras and Tensorflow.
!pip install stardist # contains tools to operate STARDIST.
!pip install edt
!pip install wget

In [1]:
from __future__ import print_function, unicode_literals, absolute_import, division
import numpy as np
import matplotlib.pyplot as plt
import os.path
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from glob import glob
from tqdm import tqdm
from tifffile import imread
from csbdeep.utils import Path, normalize
from csbdeep.io import save_tiff_imagej_compatible

from stardist import export_imagej_rois, random_label_cmap, _draw_polygons
from stardist.models import StarDist2D

Using TensorFlow backend.


# Variables

input imgs folder : containing the splitted img stack(format:tif)

use trained model : model name & model base directory(where containing all models)

output_path : where you want to save the results(it will create a folder(the last path eg:/Prediction_output) contains the results)

In [ ]:
##Connect!
from google.colab import drive
drive.mount('/content/gdrive')

In [2]:
# input imgs folder
open_path = 'V:/Marco/Wound Healing/Triangulation analysis/50%_1stcut_M197B_2'

# trained model
model_name = 'Fishn32e400s100resize5'
model_basedir = 'C:/Users/mengwei.HIVE2008/00MengPython/2DStarDist/models'

# Where you want to create output folder
output_path = 'V:/Marco/Wound Healing/Triangulation analysis/50%_1stcut_M197B_2_output'

# Load datas & model

In [3]:
complete_open_path = os.path.join(open_path,'*.tif')
X = sorted(glob(complete_open_path))
X = list(map(imread,X))

n_channel = 1 if X[0].ndim == 2 else X[0].shape[-1]
axis_norm = (0,1)   # normalize channels independently

# model = StarDist2D.from_pretrained('2D_versatile_fluo') # it is pretrained model, can try it if you want.
model = StarDist2D(None, name=model_name, basedir=model_basedir)

if os.path.exists(output_path) == False:
    os.makedirs(output_path)
os.makedirs(output_path+"/"+"/label(resize)")
os.makedirs(output_path+"/"+"/roi(resize)")
os.makedirs(output_path+"/"+"/quality")
os.makedirs(output_path+"/"+"/quality2")
save_label_path = output_path+"/"+"/label(resize)"
save_roi_path = output_path+"/"+"/roi(resize)"
save_q_path = output_path+"/"+"/quality"
save_q2_path = output_path+"/"+"/quality2"

Instructions for updating:
Colocations handled automatically by placer.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.487325, nms_thresh=0.3.


# Processing

Once it print "Finished", mission complete.

In [ ]:
%matplotlib agg
%matplotlib agg

ii = 0
for f in X:
    ii = ii + 1
    print("processing ",'{:0>4}'.format(str(ii)))
    img = normalize(X[ii-1], 1,99.8, axis=axis_norm) # image
    labels, polygons = model.predict_instances(img)
     
    # quality check
    plt.figure(figsize=(70,100))
    img_show = img if img.ndim==2 else img[...,0]
    coord, points, prob = polygons['coord'], polygons['points'], polygons['prob']
    plt.subplot(121); plt.imshow(img_show, cmap='gray'); plt.axis('off')
    a = plt.axis()
    _draw_polygons(coord, points, prob, show_dist=True)
    plt.axis(a)
    plt.title('Show nRays',fontsize=50)
    plt.subplot(122); plt.imshow(img_show, cmap='gray'); plt.axis('off')
    lbl_camp = random_label_cmap()
    plt.imshow(labels, cmap=lbl_camp, alpha=0.5)
    plt.tight_layout()
    plt.title('Labels',fontsize=50)
    complete_save_q_path = os.path.join(save_q_path,'overlay_%s.png'%'{:0>4}'.format(str(ii))) 
    # save plot
    plt.savefig(complete_save_q_path)
    plt.show()
        
    # save label, roi
    complete_save_label_path = os.path.join(save_label_path,'img_labels_%s.tif'%'{:0>4}'.format(str(ii)))  
    save_tiff_imagej_compatible(complete_save_label_path, labels, axes='YX')
    complete_save_roi_path = os.path.join(save_roi_path,'img_rois_%s.zip'%'{:0>4}'.format(str(ii)))  
    export_imagej_rois(complete_save_roi_path, polygons['coord'])
print("Finished")

processing  0001


C:\Users\mengwei.HIVE2008\anaconda3\envs\TFgpu\lib\site-packages\ipykernel_launcher.py:28: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
C:\Users\mengwei.HIVE2008\anaconda3\envs\TFgpu\lib\site-packages\csbdeep\io\__init__.py:40: UserWarning: Converting data type from 'int32' to ImageJ-compatible 'int16'.
  warnings.warn("Converting data type from '%s' to ImageJ-compatible '%s'." % (t, np.dtype(t_new)))


processing  0002


C:\Users\mengwei.HIVE2008\anaconda3\envs\TFgpu\lib\site-packages\ipykernel_launcher.py:28: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
C:\Users\mengwei.HIVE2008\anaconda3\envs\TFgpu\lib\site-packages\csbdeep\io\__init__.py:40: UserWarning: Converting data type from 'int32' to ImageJ-compatible 'int16'.
  warnings.warn("Converting data type from '%s' to ImageJ-compatible '%s'." % (t, np.dtype(t_new)))


processing  0003


...

C:\Users\mengwei.HIVE2008\anaconda3\envs\TFgpu\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  if sys.path[0] == '':
C:\Users\mengwei.HIVE2008\anaconda3\envs\TFgpu\lib\site-packages\ipykernel_launcher.py:28: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
C:\Users\mengwei.HIVE2008\anaconda3\envs\TFgpu\lib\site-packages\csbdeep\io\__init__.py:40: UserWarning: Converting data type from 'int32' to ImageJ-compatible 'int16'.
  warnings.warn("Converting data type from '%s' to ImageJ-compatible '%s'." % (t, np.dtype(t_new)))


processing  0027


C:\Users\mengwei.HIVE2008\anaconda3\envs\TFgpu\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  if sys.path[0] == '':
C:\Users\mengwei.HIVE2008\anaconda3\envs\TFgpu\lib\site-packages\ipykernel_launcher.py:28: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
C:\Users\mengwei.HIVE2008\anaconda3\envs\TFgpu\lib\site-packages\csbdeep\io\__init__.py:40: UserWarning: Converting data type from 'int32' to ImageJ-compatible 'int16'.
  warnings.warn("Converting data type from '%s' to ImageJ-compatible '%s'." % (t, np.dtype(t_new)))


processing  0028


C:\Users\mengwei.HIVE2008\anaconda3\envs\TFgpu\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  if sys.path[0] == '':
